# Tutorial 07-02 Transforming and Loading Data

#### 1. Repeat Extract Logic from Tutorial 07-01

In the previous tutorial, we extracted data from an API and converted it into a pandas DataFrame.  Now we can put some transformations in place and put everything together.  

While we're copying our code over this time, let's take an opportunity to make our code a bit more reusable.  In the earlier chapters of this book, we brought up the concept of functional programming.  Now we can put that concept into practice.  


The one big thing we can abstract away is the extraction of data and conversion to a DataFrame.  There wasn't anything specific to our use-case in that (aside from the specific URL we formatted).  All that logic about getting a response from the URL, converting to Python data types, then ultimately to a DataFrame is totally repeatable for other URLs.  Let's turn that into a function we can use here and reuse elsewhere.

In [ ]:
def extract_to_df(url):
    '''
    TODO - write docstring
    '''

    # Request data from the API
    response = requests.get(url)

    # Convert the response to JSON
    response_text = response.text

    # Convert JSON to a Python data types (lists/dictionaries)
    results_list = json.loads(response_text)

    # Convert the Python lists/dictionaries to a Pandas DataFrame
    df = pandas.DataFrame(results_list)

    # Return the DataFrame
    return df


The majority of the remaining logic we had was specific to our use case and might not be worth abstracting away into a function at this point.  The repeatable functionality we turned into a function will make our code a bit cleaner though.  Notice the last line of our code block below calls the function.

In [ ]:
import pandas
import requests
import json
import datetime
import arcgis

# base url for the data
url = "https://data.sfgov.org/resource/vw6y-z8j6.json"

# build the field list
field_list = [
    'service_request_id',
    'requested_datetime',
    'status_notes',
    'service_name',
    'service_subtype',
    'lat',
    'long',
    'neighborhoods_sffind_boundaries',
    'source',
    'supervisor_district',
    'media_url',
    'point'
]
fields = "$select=" + ",".join(field_list)

# build the where statement
now = datetime.datetime.now()
start_date = now - datetime.timedelta(days=7)

# Set the time to midnight
midnight = datetime.time()

# Combine the date and time
start_day_midnight = datetime.datetime.combine(start_date, midnight)
start_day_string = start_day_midnight.strftime('%Y-%m-%dT%H:%M:%S.%f') 
where = f"$where=status_description='Open' and requested_datetime > '{start_day_string}'"

# set a record limit
limit = "&$limit=10000"

# combine the url components
full_url = url+"?" + fields + "&" + where + limit

# call our function to extract and convert to a DataFrame
df = extract_to_df(full_url)

In [ ]:
print(df.shape)
df.head()

#### 2.  Condition the Data

Let's get our data ready to summarize.  If we want to summarize counts of records with specific service name values, we can add new columns to the raw data to summarize.  We'll use a common pattern combining a pandas method and a core Pandas concept called a *lambda function*.

Lambda functions are short one-line functions that we can use to execute relatively simple logic without having to define a function.  Let's take the following example of a function.

We've defined a function that returns a 1 if it receives the specific string "Street and Sidewalk Cleaning".  If it receives anything else, it returns a 0.  This is going to be really useful for summary purposes later.  We'll have to do this multiple times though, so writing it in a more concise way would be helpful.  This is where the lambda function comes in.

In [ ]:
def street_cleaning(value):
    if value == 'Street and Sidewalk Cleaning':
        return 1
    else:
        return 0
    
print(street_cleaning('Street and Sidewalk Cleaning'))
print(street_cleaning('some other string'))

In [ ]:
lambda_func = lambda x: 1 if x == 'Street and Sidewalk Cleaning' else 0

print(lambda_func('Street and Sidewalk Cleaning'))
print(lambda_func('some other string'))

This gave us the same values as the more verbose function we defined previously, but was defined all in one line.  Now we can combine that with a pandas method called **apply** that applies our function to all the values in a column (or Series in pandas terminology).

In [ ]:
df['service_name'].apply(
    lambda r: 1 if r== 'Street and Sidewalk Cleaning' else 0
)

Note that this returns another column.  We can now save this new column as a column in our DataFrame.  We can also do this with the other string values we're interested in.

In [ ]:
df['street_sidewalk_cleaning'] = df['service_name'].apply(
    lambda r: 1 if r== 'Street and Sidewalk Cleaning' else 0
)

df['graffiti'] = df['service_name'].apply(
    lambda r: 1 if r== 'Graffiti' else 0
)

df['total_cases'] = 1

In [ ]:
df[['service_name','street_sidewalk_cleaning','graffiti','total_cases']]

#### 3. Summarize By Neighborhood

Now that we've gotten our raw data directly from the source, we can continue our transformation by summarizing the data by neighborhood.  This will allow us to join with the spatial data going forward.

In [ ]:
df_neighborhood = df.groupby("neighborhoods_sffind_boundaries").agg(
    {
        "total_cases": "sum",
        "street_sidewalk_cleaning": "sum",
        "graffiti": "sum"
    }
)
df_neighborhood

#### 4.  Read Spatial Data and Merge with Summary Data

Now let's create a Spatially Enabled DataFrame using neighborhood data.

In [ ]:
sedf_neighborhoods = pandas.DataFrame.spatial.from_featureclass(
    "../Chapter 06/Tutorial_06_02.gdb/SF_Find_Neighborhoods"
)

We're going to use a pandas method called **merge** here.  We discuss this operation in the previous chapter, but it's basically pandas' version of a table join.  We're going to join our geometry with our summarized data and create a new DataFrame.

In [ ]:
sedf_merge = sedf_neighborhoods.merge(
    df_neighborhood, 
    how = 'inner', 
    left_on = 'name', 
    right_on = 'neighborhoods_sffind_boundaries'
)

#### 5.  Write the data to an output

The "Load" portion of an ETL process can take several forms.  We're going to choose to overwrite here and discuss other options following the exercise.

In [ ]:
sedf_merge.spatial.to_featureclass(
    "../Chapter 06/Tutorial_06_02.gdb/sf_311_cases_prev_7_days"
)